In [22]:
import kfp
from kfp.components import load_component_from_file
import kfp.dsl as dsl
from kfp.v2.dsl import Model, Dataset, Artifact

init_op = load_component_from_file("components/init_artifacts/component_SDKv2.yaml")
train_op = load_component_from_file("components/train/component_SDKv2.yaml")

@dsl.pipeline(
    name="KWS-train-test-pipe"
)
def pipeline(
    config_file_url: str,
    dataset_path: str,
    model_S3_bucket: str,
):
    init_task = init_op(
        config_path = config_file_url,
        dataset_path = dataset_path,
        bucket_name = model_S3_bucket,
        version = "v0.0.1")
    train_task = train_op(
        dataset = init_task.outputs["train_dataset"],
        config = init_task.outputs["train_config"]
    )

kfp.compiler.Compiler(mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE).compile(
    pipeline_func=pipeline,
    package_path='pipeline.yaml')


In [19]:
!pip list | grep kfp

kfp                           1.8.13
kfp-pipeline-spec             0.1.16
kfp-server-api                1.8.4


In [14]:
from kfp.v2.dsl import Input, Output, OutputPath, Dataset, Model
from kfp.v2.dsl import component
@component(
    output_component_file="component2.yaml"
)
def train(
    # Use Input[T] to get a metadata-rich handle to the input artifact of type `Dataset`.
    dataset: Input[Dataset],
    # Use Output[T] to get a metadata-rich handle to the output artifact of type `Dataset`.
    model: Output[Model],
    # An input parameter of type int.
    num_steps: int,
    # An output parameter of type str.
    output_message_path: OutputPath(str),
):
    """Dummy Training step."""

    with open(dataset.path, 'r') as input_file:
        dataset_contents = input_file.read()

    with open(model.path, 'w') as output_file:
        for i in range(num_steps):
            output_file.write(f'Step {i}\n=====\n')

    # Model artifact has a `.metadata` dictionary
    # to store arbitrary metadata for the output artifact.
    model.metadata['accuracy'] = 0.9

    with open(output_message_path, 'w') as output_file:
        output_file.write('Model trained successfully.')

# import kfp
# kfp.components.create_component_from_func(
#     func= train,
#     output_component_file="test.yaml"
# )

In [8]:
!pip install kfp==2.0.0b3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.3/334.3 kB 886.6 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for kfp: filename=kfp-2.0.0b3-py3-none-any.whl size=476367 sha256=4fbeda29bcf00536e4ba3b37ba39ff1b03b1dd742adc3e6fc8834f5d002b9d14
  Stored in directory: /home/lebugcat/.cache/pip/wheels/65/fc/20/b9dc05d38381c134023598e898871f5de2047ad438e7f7f93f
Successfully built kfp
  Attempting uninstall: kfp
    Found existing installation: kfp 2.0.0b2
    Uninstalling kfp-2.0.0b2:
      Successfully uninstalled kfp-2.0.0b2
